In [1]:
from collections import defaultdict
from tqdm.notebook import tqdm, trange
from typing import NamedTuple, TypedDict, Literal, NotRequired
from tba_session import session, ROOT_URL

In [2]:
YEAR = 2025

In [3]:
class TeamEventOPR(NamedTuple):
    team: str
    event: str
    opr: float
    week: int
    regional_points: int

class DistrictList(TypedDict):
    abbreviation: str
    display_name: str
    key: str
    year: int

class Webcast(TypedDict):
    type: Literal["youtube", "twitch", "ustream",
                  "iframe", "html5", "rtmp", "livestream", "direct_link",
                  "mms", "justin", "stemtv", "dacast"]
    channel: str
    date: str | None
    file: str | None

class Event(TypedDict):
    key: str
    name: str
    event_code: str
    event_type: int
    district: DistrictList | None
    city: str | None
    state_prov: str | None
    country: str | None
    start_date: str
    end_date: str
    year: int
    short_name: str | None
    event_type_string: str
    week: int
    address: str | None
    postal_code: str | None
    gmaps_place_id: str | None
    gmaps_url: str | None
    lat: float | None
    lng: float | None
    location_name: str | None
    timezone: str
    website: str | None
    first_event_id: str | None
    first_event_code: str | None
    webcasts: list[Webcast]
    division_keys: list[str]
    parent_event_key: str | None
    playoff_type: int | None
    playoff_type_string: str | None

class Team(TypedDict):
    key: str
    team_number: int
    nickname: str
    name: str
    school_name: str | None
    city: str | None
    state_prov: str | None
    country: str | None
    address: str | None
    postal_code: str | None
    gmaps_place_id: str | None
    gmaps_url: str | None
    lat: float | None
    lng: float | None
    location_name: str | None
    website: NotRequired[str | None]
    rookie_year: int | None

class PointsAtEventForTeam(TypedDict):
    total: int
    alliance_points: int
    elim_points: int
    award_points: int
    qual_points: int

class TiebrakersAtEventForTeam(TypedDict):
    highest_qual_scores: list[int]
    qual_wins: list[int]

class DistrictPoints(TypedDict):
    points: dict[str, PointsAtEventForTeam]
    tiebreakers: dict[str, TiebrakersAtEventForTeam]

class OPRs(TypedDict):
    oprs: dict[str, float]
    dprs: dict[str, float]
    ccwms: dict[str, float]

In [4]:
EVENTS: list[Event] = session.get(ROOT_URL + f"/events/{YEAR}").json()

deep_climbs = defaultdict(int)
shallow_climbs = defaultdict(int)
parks = defaultdict(int)
nones = defaultdict(int)

skipped = 0

did_not_meet = 0
met_but_would_not = 0
met_and_would = 0

trough_count: dict[str, tuple[int, str, str, str]] = {}

events_by_code = {event["key"]: event for event in EVENTS}

for event in tqdm(EVENTS):
    for match in session.get(
        f"{ROOT_URL}/event/{event['key']}/matches"
    ).json():
        if match["score_breakdown"] is None:
            skipped += 1
            continue
        for alliance in ("red", "blue"):
            shallow_climb_count = 0
            deep_climb_count = 0
            for i in range(3):
                climb = match["score_breakdown"][alliance]["endGameRobot" + str(i + 1)]
                team_key = match["alliances"][alliance]["team_keys"][i]
                nones[team_key] += climb == "None"
                parks[team_key] += climb == "Parked"
                shallow_climbs[team_key] += climb == "ShallowCage"
                deep_climbs[team_key] += climb == "DeepCage"
                shallow_climb_count += climb == "ShallowCage"
                deep_climb_count += climb == "DeepCage"
            if match["comp_level"] == "qm":
                barge_points = match["score_breakdown"][alliance]["endGameBargePoints"]
                did_not_meet += barge_points < 14
                met_but_would_not += 14 <= barge_points < 16
                met_and_would += barge_points >= 16
            trough_count[alliance + " " + match["key"]] = (
                # match["score_breakdown"][alliance]["autoReef"]["trough"]
                # + match["score_breakdown"][alliance]["teleopReef"]["trough"],
                # match["score_breakdown"][alliance]["wallAlgaeCount"],
                + match["score_breakdown"][alliance]["netAlgaeCount"],
                *match["alliances"][alliance]["team_keys"],
            )
            # if shallow_climb_count >= 1 and deep_climb_count >= 1:
            #     print(match["key"], alliance, deep_climb_count, "deep climb" + ("" if deep_climb_count == 1 else "s"), "and", shallow_climb_count, "shallow climb" + ("" if shallow_climb_count == 1 else "s"))

ACTUAL_WORLDS_TEAMS = set()

for event in filter(
    lambda event: event["event_type"] == 3 or event["event_type"] == 4, EVENTS
):
    ACTUAL_WORLDS_TEAMS.update(
        session.get(
            ROOT_URL + f"/event/{event['key']}/teams/keys"
        ).json()
    )

  0%|          | 0/222 [00:00<?, ?it/s]

In [5]:
for team_key, count in shallow_climbs.items():
    if count > 0 and team_key in ACTUAL_WORLDS_TEAMS:
        print(f"{team_key}: {count} shallow climbs")

frc5216: 11 shallow climbs
frc1511: 7 shallow climbs
frc10466: 16 shallow climbs
frc3880: 1 shallow climbs
frc973: 1 shallow climbs
frc4680: 10 shallow climbs
frc2200: 12 shallow climbs
frc10157: 24 shallow climbs


In [6]:
met_and_would, met_and_would / (met_and_would + did_not_meet + met_but_would_not)

(8475, 0.28591188178935295)

In [7]:
for alliance_key, count, teams in sorted(
    [
        (alliance_key, count[0], count[1:])
        for alliance_key, count in trough_count.items()
    ],
    key=lambda x: x[1],
    reverse=True,
)[:13]:
    event_key = alliance_key.split(" ")[1].split("_")[0]
    match_key = alliance_key.split(" ")[1].split("_")[1]
    event = events_by_code[event_key]
    print(
        f"{alliance_key.split(' ')[0].capitalize()}\t{', '.join(team[3:] for team in teams)}\t{count}\t[{'Playoffs ' + match_key[2:-2] if match_key.startswith('sf') else 'Finals ' + match_key[-1] if match_key.startswith('f') else 'Quals ' + match_key[2:]} at {event['name']}](https://thebluealliance.com/match/{alliance_key.split(' ')[1]})"
    )

Blue	2470, 3100, 1816	18	[Playoffs 10 at Minnesota North Star Regional](https://thebluealliance.com/match/2025mnum_sf10m1)
Red	1731, 686, 620	17	[Playoffs 6 at FIRST Chesapeake District Championship presented by Qualcomm](https://thebluealliance.com/match/2025chcmp_sf6m1)
Red	8612, 5535, 4004	16	[Playoffs 9 at FIM District Ferris State Event presented by Ferris State](https://thebluealliance.com/match/2025mibig_sf9m1)
Blue	102, 5401, 1391	16	[Quals 46 at FIRST Mid-Atlantic District Championship](https://thebluealliance.com/match/2025mrcmp_qm46)
Red	9654, 112, 10435	15	[Quals 26 at Midwest Regional](https://thebluealliance.com/match/2025ilch_qm26)
Blue	9431, 4485, 3865	15	[Quals 12 at FIN District Lafayette Event](https://thebluealliance.com/match/2025inlaf_qm12)
Red	3100, 5903, 1619	15	[Playoffs 2 at Lake Superior Regional](https://thebluealliance.com/match/2025mndu_sf2m1)
Blue	3100, 3871, 7019	15	[Quals 6 at Minnesota North Star Regional](https://thebluealliance.com/match/2025mnum_qm6

In [8]:
TEAMS: dict[str, Team] = {
    team["key"]: team
    for team in session.get(
        ROOT_URL + "/district/2025ne/teams"
    ).json()
}
RANKINGS: dict[str] = {
    team["team_key"]: team
    for team in session.get(
        ROOT_URL + "/district/2025ne/rankings"
    ).json()
}

MA_PRE_DCMP: dict[str, int] = {}
MA_TOTAL: dict[str, int] = {}
for key, team in TEAMS.items():
    if team["state_prov"] != "Connecticut":
        continue
    MA_TOTAL[key] = RANKINGS[key]["point_total"]
    MA_PRE_DCMP[key] = RANKINGS[key]["point_total"] - sum(
        event["total"]
        for event in RANKINGS[key]["event_points"]
        if event["district_cmp"]
    )

MA_RANK: dict[str, int] = {team_key: i for i, team_key in enumerate(sorted(MA_TOTAL, key=MA_TOTAL.get, reverse=True), 1)}

In [9]:
for i, team_key in enumerate(sorted(MA_PRE_DCMP, key=MA_PRE_DCMP.get, reverse=True), 1):
    team = TEAMS[team_key]
    print(
        f"{i:<3}| {MA_RANK[team_key]:<3}| {team['team_number']:<6}| {team['nickname']:<30}| {MA_PRE_DCMP[team_key]:<4}| {MA_TOTAL[team_key]}"
    )

1  | 2  | 195   | CyberKnights                  | 153 | 276
2  | 1  | 7407  | Wired Boars                   | 146 | 341
3  | 3  | 176   | Aces High                     | 140 | 227
4  | 5  | 5142  | RoboDominators                | 104 | 212
5  | 6  | 230   | Gaelhawks                     | 102 | 177
6  | 13 | 2064  | The Panther Project           | 95  | 122
7  | 10 | 571   | Paragon Robotics              | 92  | 137
8  | 4  | 7153  | Aetos Dios (Eagles of Zeus)   | 87  | 213
9  | 12 | 175   | Buzz Robotics                 | 86  | 131
10 | 7  | 3182  | Athena's Warriors             | 79  | 175
11 | 14 | 155   | The TechnoNuts                | 73  | 115
12 | 8  | 1699  | Robocats                      | 71  | 170
13 | 9  | 8085  | MOJO                          | 68  | 152
14 | 15 | 228   | GUS                           | 66  | 111
15 | 22 | 1991  | The Dragons                   | 60  | 81
16 | 18 | 3146  | GRANBY GRUNTS                 | 60  | 102
17 | 11 | 177   | Bobcat Robotics        